In [1]:
# Boiler Plate
import findspark
findspark.init('/home/michael/spark-2.1.0-bin-hadoop2.7/')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [5]:
from pyspark.ml.classification import LogisticRegression

In [6]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [7]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [8]:
my_log_reg_model = LogisticRegression()

In [10]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [11]:
log_summary = fitted_logreg.summary

In [13]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



Does this label match our actual prediction for it?

In [14]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

The concept of evaluators... Split into a training and testing set

In [16]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

Let's only fit on the training data

In [17]:
final_model = LogisticRegression()

In [18]:
fit_final = final_model.fit(lr_train)

In [19]:
# Evaluate on the test set
prediction_and_labels = fit_final.evaluate(lr_test)

In [20]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[9.15283140878675...|[0.99989409174130...|       0.0|
|  0.0|(692,[121,122,123...|[14.8174486775816...|[0.99999963283299...|       0.0|
|  0.0|(692,[122,123,148...|[9.81390270339615...|[0.99994531698386...|       0.0|
|  0.0|(692,[123,124,125...|[18.8962967083324...|[0.99999999378497...|       0.0|
|  0.0|(692,[124,125,126...|[28.0923872568279...|[0.99999999999936...|       0.0|
|  0.0|(692,[126,127,128...|[26.2323717497512...|[0.99999999999595...|       0.0|
|  0.0|(692,[127,128,129...|[22.1994736756292...|[0.99999999977149...|       0.0|
|  0.0|(692,[152,153,154...|[15.7643225446708...|[0.99999985755682...|       0.0|
|  0.0|(692,[153,154,155...|[8.82383237359632...|[0.99985283844278...|       0.0|
|  0.0|(692,[154

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [22]:
my_eval = BinaryClassificationEvaluator()

In [25]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [26]:
# Predicted everything perfectly... Probably pretty unrealistic
my_final_roc

1.0